In [2]:
!pwd

/home/mhossai5/Ag-Ab-Affinity2/LLM+Graph


In [3]:
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5EncoderModel
import torch
from tqdm.auto import tqdm
tqdm.pandas()

2025-11-19 16:03:41.195151: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 16:03:41.232299: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-19 16:03:41.232338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-19 16:03:41.234602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-19 16:03:41.247174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

[2025-11-19 16:03:42,744] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: warning: libstdc++.so.6, needed by /home/mhossai5/local/cuda-11.8/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::ostream::tellp()@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.so: undefined reference to `std::string::substr(unsigned long, unsigned long) const@GLIBCXX_3.4'
/home/mhossai5/.conda/envs/llm_tor2/compiler_compat/ld: /home/mhossai5/local/cuda-11.8/lib64/libcufile.

In [4]:
import re
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print("Using device: {}".format(device))

transformer_link = "Rostlab/prot_t5_xl_half_uniref50-enc"
print("Loading: {}".format(transformer_link))
T5model = T5EncoderModel.from_pretrained(transformer_link)
T5model.full() if device=='cpu' else T5model.half() # only cast to full-precision if no GPU is available
T5model = T5model.to(device)
T5model = T5model.eval()
tokenizer = T5Tokenizer.from_pretrained(transformer_link, do_lower_case=False )

Using device: cuda:0
Loading: Rostlab/prot_t5_xl_half_uniref50-enc


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
df = pd.read_csv("../ppi/data/LR_PPI.csv")

In [6]:
def embedding_dict_values(sequences):
    sequence = " ".join(list(re.sub(r"[UZOB]", "X", sequences)))
    ids = tokenizer.encode_plus(sequence, add_special_tokens=True, padding="longest")
    input_ids = torch.tensor(ids['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = T5model(input_ids=input_ids, attention_mask=attention_mask)
    token_embeddings = embedding.last_hidden_state.squeeze(0) 
    mask = attention_mask.squeeze(0).bool()  
    filtered_embeddings = token_embeddings[mask]
    return filtered_embeddings

In [7]:
max_seq = 1500
df = df[(df['Protein_1_Seq'].str.len()<=max_seq) & (df['Protein_2_Seq'].str.len()<=max_seq)]# .groupby('label',group_keys =False).apply(lambda x: x.sample(4000,replace=False))
df.shape

(65849, 6)

In [8]:
seq_lists =pd.unique(df[['Protein_1_Seq','Protein_2_Seq']].values.ravel())

In [9]:
import h5py
import numpy as np
from tqdm import tqdm

seq_lists = pd.unique(df[['Protein_1_Seq','Protein_2_Seq']].values.ravel())
file_path = "../ppi/data/LR_PPI-ProtT5-Full.h5"

with h5py.File(file_path, 'w') as hf:
    for seq in tqdm(seq_lists):
        emb = embedding_dict_values(seq).detach().cpu().numpy().ravel()
        hf.create_dataset(seq, data=emb, compression="gzip")


100%|██████████| 9649/9649 [10:08<00:00, 15.85it/s]


In [ ]:
# df['Protein_1_Seq_Emb'] = df['Protein_1_Seq'].progress_apply(lambda x: data_emb.get(x))
# df['Protein_2_Seq_Emb'] = df['Protein_2_Seq'].progress_apply(lambda x:  data_emb.get(x))

In [ ]:
# df['Protein_1_Seq_Emb'] = df['Protein_1_Seq_Emb'].progress_apply(lambda x:x.reshape(-1).detach().cpu().numpy().ravel().tolist())
# df['Protein_2_Seq_Emb'] = df['Protein_2_Seq_Emb'].progress_apply(lambda x:x.reshape(-1).detach().cpu().numpy().ravel().tolist())

In [ ]:
# df.to_parquet("../ppi/data/LR_PPI-ProtT5-Full.parquet", index =False)